In [ ]:
from datetime import date, datetime
import gzip
import os
import sys

from lxml import etree
import pubmed_parser as pp

In [ ]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from db_api.models import Article
from db_api.database import global_init, session_scope

In [ ]:
input_file_path = "pubmed20n0477.xml.gz"

In [ ]:
with gzip.open(input_file_path, "rb") as input_file:
    tree = etree.parse(input_file)

In [ ]:
root = tree.getroot()

In [ ]:
for i, element in enumerate(root.iter()):
    print("%s - %s" % (element.tag, element.text))
    if i >= 10:
        break

In [ ]:
dict_out = pp.parse_medline_xml(input_file_path, year_info_only=False, author_list=False,
    reference_list=True)

In [ ]:
len(dict_out)

In [ ]:
ad = dict_out[10]

In [ ]:
ad

In [ ]:
article = Article()

In [ ]:
article.id = "PMID:" + ad["pmid"]
article.version = "v1"
article.source = "PubMed"
article.journal = ad["journal"]
article.article_type = "postprint"
article.title = ad["title"]
pubdate = ad["pubdate"]
pubdate_dashes = pubdate.count("-")
if pubdate_dashes == 2:  # format of parsed pubdate is YYYY-MM-DD
    article.publication_date = date.fromisoformat(pubdate)
elif pubdate_dashes == 1:  # format of parsed pubdate is YYYY-MM
    article.publication_date = date.fromisoformat(pubdate + "-01")
else:  # format of parsed pubdate is YYYY
    article.publication_date = date.fromisoformat(pubdate + "-01-01")
article.update_date = date.today()
article.modified_date = datetime.now()
article.link = "https://pubmed.ncbi.nlm.nih.gov/" + ad["pmid"]
article.pmid = ad["pmid"]
article.doi = ad["doi"]
article.summary = ad["abstract"]
article.full_text = ""
article.authors = [x.strip() for x in ad["authors"].split(";") if x != ""]
article.affiliations = [ad["affiliations"]]
article.language = ""
article.keywords = [x.strip() for x in ad["keywords"].split(";") if x != ""]
article.references = ad["references"]
article.tags = [x.strip() for k in ["mesh_terms", "publication_types", "chemical_list"] for x in ad[k].split(";") if x != ""]